In [1]:
import pandas as pd
import requests
import urllib


In [2]:

charts_list = "https://shazam.p.rapidapi.com/charts/list"

charts_list_headers = {
    'x-rapidapi-key': "d78ca9f758msh31ad154b2fe50a8p12fbc9jsnabb8cabd4076",
    'x-rapidapi-host': "shazam.p.rapidapi.com"
    }

charts_list_response = requests.request("GET", charts_list, headers=charts_list_headers)



In [3]:

def get_top10_json(list_id):
    url = "https://shazam.p.rapidapi.com/charts/track"

    querystring = {"locale":"en-US","listId":list_id,"pageSize":"20","startFrom":"0"}

    headers = {
        'x-rapidapi-key': "d78ca9f758msh31ad154b2fe50a8p12fbc9jsnabb8cabd4076",
        'x-rapidapi-host': "shazam.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response


def get_top10_names(top_10_json):
    tracks_dict = top_10_json['tracks']
    names = []
    for track in tracks_dict:
        names.append((track['title'] + ' - '+ track['subtitle']))
    return names


In [43]:

def get_lat_lon(name):
    f = { 'name' : name}
    formatted_name = urllib.parse.urlencode(f)
    # print(formatted_name)
    response = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?access_token=pk.eyJ1IjoiaGFyc2gtaiIsImEiOiJja3JraHRmMnEzbnA1MndwOGI2OTY1enNrIn0.Lrx1G8lFIKLt_7OsC6ow7g".format(formatted_name[5:]))
    return response.json()['features'][0]['bbox'][1], response.json()['features'][0]['bbox'][0]


In [51]:

country_list = []
city_list = []
top10 = []
coords_lat = []
coords_lon = []

for country_dict in charts_list_response.json()['countries']:
    for city in country_dict['cities']:
        if len(city_list) < 4:
            lat, lon = get_lat_lon(city['name'])
            country_list.append(country_dict['name'])
            city_list.append(city['name'])
            coords_lat.append(lat)
            coords_lon.append(lon)
            top_10_json = get_top10_json(city['listid']).json()
            top10.append(get_top10_names(top_10_json))
            print(city_list)
        else:
            break

['Berlin']
['Berlin', 'Essen']
['Berlin', 'Essen', 'Munich']
['Berlin', 'Essen', 'Munich', 'Hamburg']


In [54]:
daily_df = pd.DataFrame({'country': country_list, 'city': city_list,"lat": coords_lat, 'lon': coords_lon ,'top10': top10})
daily_df.to_csv('./shazam.csv')

In [55]:
df = pd.read_csv('/Users/harshjhunjhunwala/Desktop/github/spotify/data/shazam.csv')

In [76]:
for row in df.iterrows():
    print(row[1][3], row[1][4])

52.3382670008426 13.0883590415111
51.34746 6.894457
48.061596 11.360781
53.3949960146367 8.30956670062143


In [82]:

import pandas as pd
from pymongo import MongoClient
import json


def csv_transfer(csv_path='/Users/harshjhunjhunwala/Desktop/github/spotify/data/shazam.csv', db_name='spotty', coll_name='csv_import', db_url='mongodb://localhost:27017/', db_port=27017):
    """ 
    sending the 'world trends' csv file to mongo_db. This should run once a day to keep the data updated.
    
    Imports a csv file at path csv_name to a mongo colection
    returns: count of the documants in the new collection
    """
    client = MongoClient(db_url, db_port)
    db = client[db_name]
    coll = db[coll_name]
    data = pd.read_csv(csv_path)
    payload = json.loads(data.to_json(orient='records'))
    coll.remove()
    coll.insert(payload)
    return str(coll.count())

In [83]:
csv_transfer

<function __main__.csv_transfer(csv_path='/Users/harshjhunjhunwala/Desktop/github/spotify/data/shazam.csv', db_name='spotty', coll_name='csv_import', db_url='mongodb://[Harsh420:A/asdfgh09@]host1[:27017]]', db_port=27017)>

In [6]:

# for country_dict in charts_list_response.json()['countries']:
#     if country_dict['name'] == 'India':
#         for city in country_dict['cities']:
#             print(city)
#             break
#             if len(city_list) < 4:
#                 country_list.append('India')
#                 city_list.append(city['name'])
#                 top_10_json = get_top10_json(city['listid']).json()
#                 top10.append(get_top10_names(top_10_json))
#                 print(city_list)
#             else:
#                 break
#         break

{'id': '1273294', 'name': 'Delhi', 'countryid': 'IN', 'listid': 'ip-city-chart-1273294'}
